In [1]:
import pandas as pd
import datasets

In [2]:
dartdataset = datasets.load_dataset('nlpHakdang/beneficiary',  data_files = "dart_ver1_2.csv", use_auth_token='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')

Using custom data configuration nlpHakdang___beneficiary-53bfe0868077b71a
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___beneficiary-53bfe0868077b71a/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
newsdataset = datasets.load_dataset('nlpHakdang/aihub-news30k',  data_files = "news_train_1_0.csv", use_auth_token='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')

Using custom data configuration nlpHakdang___aihub-news30k-e2d24441e999a39b
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___aihub-news30k-e2d24441e999a39b/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# news_data = pd.read_csv("../data/news/train/MainData.csv")
# #data2 = pd.read_csv("../data/news/train/SubData.csv")

In [ ]:
# # Dart load 하기
# # with open("corp_2500_result.pickle", "rb") as fp:   # Unpickling
# #   dart_data = pickle.load(fp)
# dart = pd.read_csv('../data/dart/dart_v1_1_remove_context.csv')

In [34]:
dart_dict= {
    "corp_name" :[],
    "corp_text":[]
}

In [35]:
contents_list = ['회사의 개요', '회사의 연혁', '사업의 개요', '주요 제품 및 서비스', '주요계약 및 연구개발활동']
contents_list = ['회사의 개요',  '사업의 개요', '주요 제품 및 서비스','주요계약 및 연구개발활동']
# contents_list = ['회사의 개요',  '사업의 개요', '주요 제품 및 서비스']

In [36]:
contents_list

['회사의 개요', '사업의 개요', '주요 제품 및 서비스', '주요계약 및 연구개발활동']

In [37]:
for corp in dartdataset['train']:
    dart_text = ""
    dart_dict['corp_name'].append(corp['corp_name'])
    for content in contents_list:
        dart_text += corp[content]
    dart_dict['corp_text'].append(dart_text)
    

In [106]:
length = []
for corp in dart_dict['corp_text']:
    length.append(len(corp))

In [99]:
import numpy as np

In [100]:
#['회사의 개요',  '사업의 개요', '주요 제품 및 서비스']
max(length),min(length),np.mean(length), np.median(length)

(64339, 497, 7392.284728213977, 5176.0)

In [107]:
#['회사의 개요',  '사업의 개요', '주요 제품 및 서비스','주요계약 및 연구개발활동']
max(length),min(length),np.mean(length), np.median(length)

(92699, 513, 11478.55823986195, 8265.5)

In [9]:
import re 
def preprocessing(s): 
    hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')
    result = hangul.sub('', s)
    return result

In [10]:
pre_dart = {}
for idx in range(len(dart_dict['corp_name'])):
    pre_dart[dart_dict['corp_name'][idx]] = preprocessing(dart_dict['corp_text'][idx])

In [11]:
pre_name = {}
for idx in range(len(dart_dict['corp_name'])):
    pre_name[preprocessing(dart_dict['corp_text'][idx])] = dart_dict['corp_name'][idx]

In [12]:
from tqdm import tqdm

In [13]:
news_list = []
for news in tqdm(newsdataset['train']):
    if news['category'] == '경제' or news['category'] =='기업':
        news_list.append(news['article'])

100%|██████████| 271093/271093 [00:40<00:00, 6725.96it/s]


In [14]:
pre_news = [preprocessing(txt) for txt in news_list]

- 의미가 존재 [29,36,38]
- 언급되면 표시해줄 것 [42]
- 제주도나오면 반응 [45]
- 엔터[46]
- 농장[200]

In [111]:
len(pre_news)

27303

In [15]:
split_news=[]
for article in news_list:
    split_news.append(list(map(preprocessing,article.split("."))))

In [16]:
split_news=[]
for article in news_list:
    split_news.append(article.split("."))

In [17]:
from konlpy.tag import Mecab
tokenizer = Mecab()

In [112]:
tokenizer.nouns("안녕하세요 여러분")

['안녕', '여러분']

In [18]:
tot = []
for txt in tqdm(list(pre_dart.values())):
    tot.append(tokenizer.nouns(txt))

100%|██████████| 2318/2318 [00:42<00:00, 53.91it/s]


In [19]:
from rank_bm25 import BM25Okapi
from krwordrank.word import summarize_with_keywords
bm25 = BM25Okapi(tot)

In [ ]:
def print_corp(newsindex, split_news, dart_dict, bm25, pre_dart, pre_name):
    target_idx=newsindex
    target = split_news[target_idx]
    inside_corp = []

    stopwords=['모두','지난','있다','회의','강조','성과','오는','모집','과정','계약',"체결","건립",'현대식','사업',
            '주관사로','통해','일까지','이벤트','지원','지역','기자','실험','한다','진행','설명회','예정이다',
            '이번','방문','올해','제공','찾아가','재림','당시','한국','프로젝트','있는','추진','다양한','적극',
            '위해','나갈','협력','협약','상당의','물품','명으로','지난해','증가했다','의한','영향','보면','통계',
            '순위','명당','관련','업체','부실','관심','명칭','합작법인','일자리','시흥배곧','서울대','했다',
            '한다고','위원장은','기반','혁신','인재','마련','정책','권고안','함께','시장','성장','대비','전월',
            '전년','동월','물가','하락','소비자','상승','운영','증가','억원','억만원','할인','사용','것으']
    keywords = summarize_with_keywords(target, min_count=4, max_length=7,
        beta=0.85, max_iter=10,stopwords=stopwords,verbose=True)
    tokenized_query = list(keywords.keys())

    for key in ' '.join(target).split(" "):
        if key in dart_dict['corp_name']:
            inside_corp.append(key)

    for key in tokenized_query:
        if key in dart_dict['corp_name']:
            inside_corp.append(key)
    
    inside_corp = list(set(inside_corp))
    print("키워드 :",tokenized_query)
    
    doc_scores = bm25.get_scores(tokenized_query)
    if max(doc_scores) <3:
        print("max :",max(doc_scores))
        print("적절한 회사없음")
    else:
        result = []
        rank=1
        list_dart_n = bm25.get_top_n(tokenized_query, list(pre_dart.values()), n=3)
        print("타겟뉴스")
        print(" ".join(target))
        for dart in list_dart_n:
            print(f"{rank}순위 기업 : {pre_name[dart]}")
            print(f"사업보고서 ::")
            print(dart[:2000])
            rank+=1

In [21]:
dart_dict['corp_name']

['엑세스바이오',
 '지오엠씨',
 '클루넷',
 '글로벌에스엠',
 '씨케이에이치',
 'GRT',
 '윙입푸드',
 '루트원플러스',
 '로스웰',
 '골든센츄리',
 '에스티씨라이프',
 '에임하이글로벌',
 '디비엘',
 '더스텔라',
 '코오롱티슈진',
 'SBI핀테크솔루션즈',
 '한국주강',
 '예스24',
 '지놈앤컴퍼니',
 '영신금속',
 '세명전기',
 '대동스틸',
 '엘비세미콘',
 '블루콤',
 '다스코',
 '조일알미늄',
 '비비안',
 '네오티스',
 '씨아이테크',
 '대림통상',
 '코다코',
 '파수',
 '정원엔시스',
 '케이씨피드',
 '웹젠',
 '우리넷',
 '마이크로디지탈',
 '한국단자공업',
 '텔코웨어',
 '유비벨록스',
 'KCI',
 '문배철강',
 '오성첨단소재',
 '디아이씨',
 '에스앤더블류',
 '와이솔',
 '나노',
 '손오공',
 '새로닉스',
 '넵튠',
 '제룡전기',
 '풍국주정',
 '이지홀딩스',
 '이지바이오',
 '팜스토리',
 '마니커',
 '자연과환경',
 '코리아써키트',
 '케이피엠테크',
 '하이골드12호',
 '교보9호스팩',
 '로보로보',
 'SM Life Design',
 '롯데관광개발',
 '부산주공',
 '교보10호기업인수목적',
 '아이비김영',
 '삼기',
 '유나이티드',
 '신원종합개발',
 '신영와코루',
 '프럼파스트',
 'SFA반도체',
 'BGF리테일',
 '위즈코프',
 'HDC현대산업개발',
 '지니언스',
 '제닉',
 '삼화전자공업',
 '녹십자',
 '녹십자홀딩스',
 '테라젠이텍스',
 '동아화성',
 '파라텍',
 '예스코홀딩스',
 '시노펙스',
 '모나리자',
 '티웨이홀딩스',
 '경인양행',
 '네오위즈홀딩스',
 '이수화학',
 '다날',
 '씨앤지하이테크',
 '삼화왕관',
 '백산',
 '이지스밸류플러스리츠',
 '엔씨소프트',
 'LG이노텍',
 '피엔에이치테크',
 '하림지주',
 '에스코넥',
 '삼성제약',


In [ ]:
"전방, 대상"

In [22]:
news_list[1211]

'‘일잡(job)고 행복잡(job)고’, 11개 기업 참가대구 수성구청(구청장 김대권)이 오는 29일 오전 11시부터 오후 5시까지 취업을 희망하는 대구시 거주 미취업 여성과 경력 단절 여성들을 위한 찾아가는 취업상담 서비스 ‘굿잡(Good job) 버스’가 수성구청을 찾아온다고 25일 밝혔다.수성여성일자리센터가 주관하는 굿잡버스는 시·공간적 제약으로 취업정보와 구직기회를 찾지 못하는 미취업자를 위해 직접 찾아가서 취업상담 및 기업체 현장면접을 제공하고자 마련됐다.이번 행사에는 지역의 11개 기업체가 직·간접적으로 참가하는데 현장면접에 2개 기업체(지혜의 숲, 대구광역자활센터)가, 간접채용에 9개 기업이 참여할 예정이다.채용 직종은 교육서비스업, 사회복지서비스업, 제조업, 노인여가복지시설업, 일반병원 및 요양병원 등이다.또 구직 여성은 전문상담사의 1:1 취업상담, 면접코칭 등 다양한 취업지원 서비스를 지원받을 수 있다.이날 오후 1시부터는 수성여성클럽이 주관하는 ‘로즈아트 플리마켓’도 함께 개최할 예정이다.플리마켓에서 수성여성클럽에서 운영하는 다양한 프로그램과 수성새일센터 직훈과정을 수료한 솜씨 있는 여성들이 셀러로 참여해 가죽공예제품, 향초, 디퓨저 등 다양한 제품을 선보일 예정이다.김대권 수성구청장은 “여성의 경제활동과 사회참여를 높이고 여성이 도시 공간의 변화에 주도적으로 참여할 수 있는 환경을 조성하는 것이야말로 진정한 여성복지라는 생각으로 구직을 희망하는 여성을 위한 다양한 일자리를 마련하겠다”고 말했다.이동률 기자 leedr@idaegu.com'

In [23]:
print_corp(1209)

scan vocabs ... 
num vocabs = 34
done = 10
키워드 : ['피해', '사료용', '농가', '보장', '보험료', '병해충']
타겟뉴스
경북지역 농가, 총 보험료의 최대 15% 수준만 부담모두 7종 병해충 및 사료용 벼 보장받아농협손해보험 경북총국이 ‘벼’ 농작물재해보험을 판매한다고 23일 밝혔다 벼 보험은 태풍, 우박, 집중호우를 비롯한 다양한 자연재해와 새나 짐승으로부터의 피해, 화재로 인한 피해 등을 보상 받는다 또 기후 변화 등으로 병해충 피해가 증가하는 것을 고려해 벼멸구, 도열병 등 기존 병해충 6종에 세균성벼알마름병을 추가해 모두 7종의 병해충도 보장한다 특히 올해부터는 한우와 육우 등의 사료로 쓰이는 사료용 벼도 농작물재해보험으로 보장받을 수 있도록 제도를 개선했다 사료용 벼는 자연재해와 새나 짐승, 화재로 인한 피해 등으로부터 안전장치가 마련된다 또 농가의 보험료 부담을 덜고자 정부가 50~60%, 지방자치단체는 30~45%, 지역농협에서 0~15% 정도의 보험료를 지원한다 경북지역 농가는 총 보험료의 0~15% 수준만 부담하면 된다 가입기간은 오는 6월28일까지다 가입은 가까운 농축협에서 가능하다 지난해 경북도 내 1만800여 농가가 벼 농작물재해보험에 가입했고 태풍과 폭염 등으로 피해를 본 2천여 농가가 41억 원의 보험금을 받았다 손동섭 농협손해보험 경북총국장은 “농작물재해보험의 보장 병충해를 늘리고 사료용 벼를 보장 품목으로 확대하는 등 농가에 실질적인 도움이 되도록 노력하고 있다”고 말했다 김종윤 기자 kjyun@idaegu com
1순위 기업 : 대유
사업보고서 ::
 회사의 개요 연결대상 종속회사 개황 단위  사  구분  연결대상회사수  주요 종속회사수  기초  증가  감소  기말  상장            비상장            합계            상세 현황은 상세표 연결대상 종속회사 현황상세 참조  연결대상회사의 변동내용 구분  자회사  사유  신규 연결          연결 제외    

In [ ]:
from krwordrank.word import summarize_with_keywords
result = summarize_with_keywords(news[23], min_count=5, max_length=10,
    beta=0.85, max_iter=10, verbose=True)

In [ ]:
news

In [ ]:
result.keys()

In [ ]:
out_list = []

In [ ]:
with open("out_list.pickle","wb") as f:
    pickle.dump(out_list,f)

In [ ]:
import pickle

In [ ]:
with open("out_list.pickle","rb") as f:
    d = pickle.load(f)

In [ ]:
d

In [ ]:
data1